<a href="https://colab.research.google.com/github/JuhiNayak/Machine-Learning/blob/master/CMPE_IoTdata_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages 
Most of them are already installed in Colab

In [ ]:
!pip install google-cloud-iot

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 501kB 7.7MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 92kB 6.0MB/s 
     |████████████████████████████████| 276kB 14.4MB/s 
     |████████████████████████████████| 122kB 17.7MB/s 
  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-cp36-none-any.whl size=18500 sha256=5440b24edadc6b870c58dc71d054ed0405b9ebac5d51c098042762ecdef28422
  Stored in directory: /root/.cache/pip/wheels/de/3a/83/77a1e18e1a8757186df834b86ce6800120ac9c79cd8ca4091b
  Created wheel for proto-plus: filename=proto_plus-1.10.1-cp36-none-any.whl size=41279 sha256=3e837130ebfb759dfb780f1f0f372f5e067787feff5faf96fe0f976f56fda205
  Stored in directory: /root/.cache/pip/wheels/08/80/3d/54dc44645ac5a646e7945bdaa67bba0816d6bcfd3aecf788f6
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=e262d21c8c4c51369d8260873

In [ ]:
!pip install google-api-python-client

In [ ]:
!pip install google-cloud-storage

In [ ]:
!pip install cryptography pyjwt paho-mqtt 

     |████████████████████████████████| 2.6MB 2.8MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.1-cp36-none-any.whl size=61544 sha256=91679541a7259aecac46643dbca5506ca27b59aa759c5c4032549be18cfee065
  Stored in directory: /root/.cache/pip/wheels/75/e2/f5/78942b19b4d135605e58dfe85fba52253b14d636aabf76904b
Successfully built paho-mqtt


In [ ]:
import paho.mqtt.client as mqtt
import jwt
import time
import ssl
import random
import os
import logging
import datetime
import argparse

# Setup Google Cloud Client

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config list

[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = juhi.nayak@sjsu.edu

Your active configuration is: [default]


In [ ]:
!gcloud config set compute/region us-central1

Updated property [compute/region].


In [ ]:
!gcloud config set compute/zone us-central1a

Updated property [compute/zone].


In [ ]:
!gcloud config list project

[core]
project (unset)

Your active configuration is: [default]


In [ ]:
!gcloud config set project 'cmpe297-hw1'

Updated property [core/project].


In [ ]:
!gcloud config list project

[core]
project = cmpe297-hw1

Your active configuration is: [default]


# Mount Google Drive

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Create a service credentials, ref: https://cloud.google.com/storage/docs/reference/libraries#client-libraries-usage-nodejs

In [ ]:
myservicecredentials='/content/gdrive/"My Drive"/"Colab Notebooks"/Googlecerts'

In [ ]:
!ls {myservicecredentials}

cmpe297-hw1-9b27f771eb2b.json  iotdevicenew  roots.pem


In [ ]:
newdevicecredentials_path=os.path.join(myservicecredentials, 'iotdevicenew')
!ls {newdevicecredentials_path}

rsa_cert.pem  rsa_private.pem


In [ ]:
myservicecredentials_path=os.path.join(myservicecredentials, 'cmpe297-hw1-9b27f771eb2b.json')

In [ ]:
!cp {myservicecredentials_path} . #copy to the local directory

In [ ]:
!cp -r {newdevicecredentials_path} .

In [ ]:
rootcredentials_path=os.path.join(myservicecredentials, 'roots.pem')
!cp {rootcredentials_path} .

In [ ]:
!ls

adc.json		       gdrive	     roots.pem
cmpe297-hw1-9b27f771eb2b.json  iotdevicenew  sample_data


In [ ]:
GOOGLE_APPLICATION_CREDENTIALS='./cmpe297-hw1-9b27f771eb2b.json'

In [ ]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

./cmpe297-hw1-9b27f771eb2b.json


# Google IoT Defined functions

In [ ]:
import paho.mqtt.client as mqtt
import jwt
import time
import ssl
import random
import os
import logging
import datetime
import argparse

from google.cloud import storage

In [ ]:
# The initial backoff time after a disconnection occurs, in seconds.
minimum_backoff_time = 1

# The maximum backoff time before giving up, in seconds.
MAXIMUM_BACKOFF_TIME = 32

# Whether to wait with exponential backoff before publishing.
should_backoff = False

def create_jwt(project_id, private_key_file, algorithm):
    """Creates a JWT (https://jwt.io) to establish an MQTT connection.
        Args:
         project_id: The cloud project ID this device belongs to
         private_key_file: A path to a file containing either an RSA256 or
                 ES256 private key.
         algorithm: The encryption algorithm to use. Either 'RS256' or 'ES256'
        Returns:
            A JWT generated from the given project_id and private key, which
            expires in 20 minutes. After 20 minutes, your client will be
            disconnected, and a new JWT will have to be generated.
        Raises:
            ValueError: If the private_key_file does not contain a known key.
        """

    token = {
        # The time that the token was issued at
        'iat': datetime.datetime.utcnow(),
        # The time the token expires.
        'exp': datetime.datetime.utcnow() + datetime.timedelta(minutes=20),
        # The audience field should always be set to the GCP project id.
        'aud': project_id
    }

    # Read the private key file.
    with open(private_key_file, 'r') as f:
        private_key = f.read()

    print('Creating JWT using {} from private key file {}'.format(
        algorithm, private_key_file))

    return jwt.encode(token, private_key, algorithm=algorithm)
# [END iot_mqtt_jwt]

In [ ]:
def error_str(rc):
    """Convert a Paho error to a human readable string."""
    return '{}: {}'.format(rc, mqtt.error_string(rc))


def on_connect(unused_client, unused_userdata, unused_flags, rc):
    """Callback for when a device connects."""
    print('on_connect', mqtt.connack_string(rc))

    # After a successful connect, reset backoff time and stop backing off.
    global should_backoff
    global minimum_backoff_time
    should_backoff = False
    minimum_backoff_time = 1


def on_disconnect(unused_client, unused_userdata, rc):
    """Paho callback for when a device disconnects."""
    print('on_disconnect', error_str(rc))

    # Since a disconnect occurred, the next loop iteration will wait with
    # exponential backoff.
    global should_backoff
    should_backoff = True


def on_publish(unused_client, unused_userdata, unused_mid):
    """Paho callback when a message is sent to the broker."""
    print('on_publish')


def on_message(unused_client, unused_userdata, message):
    """Callback when the device receives a message on a subscription."""
    payload = str(message.payload.decode('utf-8'))
    print('Received message \'{}\' on topic \'{}\' with Qos {}'.format(
        payload, message.topic, str(message.qos)))

In [ ]:
def get_client(
        project_id, cloud_region, registry_id, device_id, private_key_file,
        algorithm, ca_certs, mqtt_bridge_hostname, mqtt_bridge_port):
    """Create our MQTT client. The client_id is a unique string that identifies
    this device. For Google Cloud IoT Core, it must be in the format below."""
    client_id = 'projects/{}/locations/{}/registries/{}/devices/{}'.format(
        project_id, cloud_region, registry_id, device_id)
    print('Device client_id is \'{}\''.format(client_id))

    client = mqtt.Client(client_id=client_id)

    # With Google Cloud IoT Core, the username field is ignored, and the
    # password field is used to transmit a JWT to authorize the device.
    client.username_pw_set(
        username='unused',
        password=create_jwt(
            project_id, private_key_file, algorithm))

    # Enable SSL/TLS support.
    client.tls_set(ca_certs=ca_certs, tls_version=ssl.PROTOCOL_TLSv1_2)

    # Register message callbacks. https://eclipse.org/paho/clients/python/docs/
    # describes additional callbacks that Paho supports. In this example, the
    # callbacks just print to standard out.
    client.on_connect = on_connect
    client.on_publish = on_publish
    client.on_disconnect = on_disconnect
    client.on_message = on_message

    # Connect to the Google MQTT bridge.
    client.connect(mqtt_bridge_hostname, mqtt_bridge_port)

    # This is the topic that the device will receive configuration updates on.
    mqtt_config_topic = '/devices/{}/config'.format(device_id)

    # Subscribe to the config topic.
    client.subscribe(mqtt_config_topic, qos=1)

    # The topic that the device will receive commands on.
    mqtt_command_topic = '/devices/{}/commands/#'.format(device_id)

    # Subscribe to the commands topic, QoS 1 enables message acknowledgement.
    print('Subscribing to {}'.format(mqtt_command_topic))
    client.subscribe(mqtt_command_topic, qos=0)

    return client
# [END iot_mqtt_config]

In [ ]:
!ls

adc.json		       gdrive	     roots.pem
cmpe297-hw1-9b27f771eb2b.json  iotdevicenew  sample_data


In [ ]:
GOOGLE_APPLICATION_CREDENTIALS='./cmpe297-hw1-9b27f771eb2b.json'
ROOT_CREDENTIALS='./roots.pem'

In [ ]:
class Args:
  algorithm = 'RS256'
  ca_certs = '/content/roots.pem'
  cloud_region = 'us-central1'
  data = 'Hello there'
  device_id = 'cmpeiotdevice2'
  jwt_expires_minutes = 20
  listen_dur = 60
  message_type = 'event'
  mqtt_bridge_hostname = 'mqtt.googleapis.com'
  mqtt_bridge_port = 8883
  num_messages = 25
  private_key_file = '/content/iotdevicenew/rsa_private.pem'#/content/gdrive/"My Drive"/CurrentWork/CMPE181Sp2020/Googlecerts/cmpe181dev1/rsa_private.pem'
  project_id = 'cmpe297-hw1'
  registry_id = 'CMPEIoT1'
  service_account_json = '/content/cmpe297-hw1-9b27f771eb2b.json'#os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")
  imagefolder_path= './data'


args=Args()

In [ ]:
print(args.private_key_file)

/content/iotdevicenew/rsa_private.pem


In [ ]:
def mqtt_device_demo(args):
    """Connects a device, sends data, and receives data."""
    # [START iot_mqtt_run]
    global minimum_backoff_time
    global MAXIMUM_BACKOFF_TIME

    # Publish to the events or state topic based on the flag.
    sub_topic = 'events' if args.message_type == 'event' else 'state'

    mqtt_topic = '/devices/{}/{}'.format(args.device_id, sub_topic)

    jwt_iat = datetime.datetime.utcnow()
    jwt_exp_mins = args.jwt_expires_minutes
    client = get_client(
        args.project_id, args.cloud_region, args.registry_id,
        args.device_id, args.private_key_file, args.algorithm,
        args.ca_certs, args.mqtt_bridge_hostname, args.mqtt_bridge_port)

    # Publish num_messages messages to the MQTT bridge once per second.
    for i in range(1, args.num_messages + 1):
        # Process network events.
        client.loop()

        # Wait if backoff is required.
        if should_backoff:
            # If backoff time is too large, give up.
            if minimum_backoff_time > MAXIMUM_BACKOFF_TIME:
                print('Exceeded maximum backoff time. Giving up.')
                break

            # Otherwise, wait and connect again.
            delay = minimum_backoff_time + random.randint(0, 1000) / 1000.0
            print('Waiting for {} before reconnecting.'.format(delay))
            time.sleep(delay)
            minimum_backoff_time *= 2
            client.connect(args.mqtt_bridge_hostname, args.mqtt_bridge_port)

        payload = '{}/{}-payload-{}'.format(
            args.registry_id, args.device_id, i)
        print('Publishing message {}/{}: \'{}\''.format(
            i, args.num_messages, payload))
        # [START iot_mqtt_jwt_refresh]
        seconds_since_issue = (datetime.datetime.utcnow() - jwt_iat).seconds
        if seconds_since_issue > 60 * jwt_exp_mins:
            print('Refreshing token after {}s'.format(seconds_since_issue))
            jwt_iat = datetime.datetime.utcnow()
            client.loop()
            client.disconnect()
            client = get_client(
                args.project_id, args.cloud_region,
                args.registry_id, args.device_id, args.private_key_file,
                args.algorithm, args.ca_certs, args.mqtt_bridge_hostname,
                args.mqtt_bridge_port)
        # [END iot_mqtt_jwt_refresh]
        # Publish "payload" to the MQTT topic. qos=1 means at least once
        # delivery. Cloud IoT Core also supports qos=0 for at most once
        # delivery.
        client.publish(mqtt_topic, payload, qos=1)

        
        # Send events every second. State should not be updated as often
        time.sleep(1)
        # for i in range(0, 60):
        #     time.sleep(1)
        #     client.loop()
    # [END iot_mqtt_run]

In [ ]:
# mqtt_device_demo(args)

In [ ]:
!gcloud pubsub topics list

---
name: projects/cmpe297-hw1/topics/cmpeiotdevice1


In [ ]:
!gcloud pubsub subscriptions list

---
ackDeadlineSeconds: 10
expirationPolicy:
  ttl: 2678400s
messageRetentionDuration: 604800s
name: projects/cmpe297-hw1/subscriptions/testpubsub-subscription1
pushConfig: {}
topic: projects/cmpe297-hw1/topics/cmpeiotdevice1
---
ackDeadlineSeconds: 600
expirationPolicy: {}
messageRetentionDuration: 604800s
name: projects/cmpe297-hw1/subscriptions/gcf-function-2-pubsub-us-central1-cmpeiotdevice1
pushConfig:
  attributes:
    x-goog-version: v1
  pushEndpoint: https://1e0f9f86b9ccf848ac72f465aad0d767-dot-ucd47fbfceaf54d53p-tp.appspot.com/_ah/push-handlers/pubsub/projects/cmpe297-hw1/topics/cmpeiotdevice1?pubsub_trigger=true
topic: projects/cmpe297-hw1/topics/cmpeiotdevice1
---
ackDeadlineSeconds: 600
expirationPolicy: {}
messageRetentionDuration: 604800s
name: projects/cmpe297-hw1/subscriptions/gcf-function-3-firestore-us-central1-cmpeiotdevice1
pushConfig:
  attributes:
    x-goog-version: v1
  pushEndpoint: https://1947d647e75270b1f278e6034976cc57-dot-ucd47fbfceaf54d53p-tp.appspot.com

In [ ]:
import random
import time
import datetime
import json

def read_sensor(count):
    tempF = 20 + 0.2*count + (random.random() * 15)
    humidity = 60 + 0.3*count+ (random.random() * 20)
    temp = '{0:0.2f}'.format(tempF)
    hum = '{0:0.2f}'.format(humidity)
    sensorZipCode = 95192#"94043"
    sensorLat = 37.3382082+ (random.random() /100)#"37.421655"
    sensorLong = -121.8863286 + (random.random() /100)#"-122.085637"
    sensorLatf = '{0:0.6f}'.format(sensorLat)
    sensorLongf = '{0:0.6f}'.format(sensorLong)
    return (temp, hum, sensorZipCode, sensorLatf, sensorLongf)

def createJSON(reg_id, dev_id, timestamp, zip, lat, long, temperature, humidity):
    data = {
      'registry_id' : reg_id,
      'device_id' : dev_id,
      'timecollected' : timestamp,
      'zipcode' : zip,
      'latitude' : lat,
      'longitude' : long,
      'temperature' : temperature,
      'humidity' : humidity
    }

    json_str = json.dumps(data)
    return json_str

In [ ]:
def mqtt_device_random(args):
    """Connects a device, sends data, and receives data."""
    # [START iot_mqtt_run]
    global minimum_backoff_time
    global MAXIMUM_BACKOFF_TIME

    # Publish to the events or state topic based on the flag.
    sub_topic = 'events' if args.message_type == 'event' else 'state'

    mqtt_topic = '/devices/{}/{}'.format(args.device_id, sub_topic)

    jwt_iat = datetime.datetime.utcnow()
    jwt_exp_mins = args.jwt_expires_minutes
    client = get_client(
        args.project_id, args.cloud_region, args.registry_id,
        args.device_id, args.private_key_file, args.algorithm,
        args.ca_certs, args.mqtt_bridge_hostname, args.mqtt_bridge_port)

    # Publish num_messages messages to the MQTT bridge once per second.
    for i in range(1, args.num_messages + 1):
        # Process network events.
        client.loop()

        # Wait if backoff is required.
        if should_backoff:
            # If backoff time is too large, give up.
            if minimum_backoff_time > MAXIMUM_BACKOFF_TIME:
                print('Exceeded maximum backoff time. Giving up.')
                break

            # Otherwise, wait and connect again.
            delay = minimum_backoff_time + random.randint(0, 1000) / 1000.0
            print('Waiting for {} before reconnecting.'.format(delay))
            time.sleep(delay)
            minimum_backoff_time *= 2
            client.connect(args.mqtt_bridge_hostname, args.mqtt_bridge_port)

        currentTime = datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        (temp, hum, sensorZipCode, sensorLat, sensorLong) = read_sensor(i)
        payload = '{}/{}-payload-{}'.format(
            args.registry_id, args.device_id, i)
        payloadJSON = createJSON(args.registry_id, args.device_id, currentTime, sensorZipCode, sensorLat, sensorLong, temp, hum)

        print('Publishing message {}/: \'{}\''.format(
            i, payloadJSON))
        # [START iot_mqtt_jwt_refresh]
        seconds_since_issue = (datetime.datetime.utcnow() - jwt_iat).seconds
        if seconds_since_issue > 60 * jwt_exp_mins:
            print('Refreshing token after {}s'.format(seconds_since_issue))
            jwt_iat = datetime.datetime.utcnow()
            client.loop()
            client.disconnect()
            client = get_client(
                args.project_id, args.cloud_region,
                args.registry_id, args.device_id, args.private_key_file,
                args.algorithm, args.ca_certs, args.mqtt_bridge_hostname,
                args.mqtt_bridge_port)
        # [END iot_mqtt_jwt_refresh]
        # Publish "payload" to the MQTT topic. qos=1 means at least once
        # delivery. Cloud IoT Core also supports qos=0 for at most once
        # delivery.
        client.publish(mqtt_topic, payloadJSON, qos=1)

        
        # Send events every second. State should not be updated as often
        time.sleep(1)
        # for i in range(0, 60):
        #     time.sleep(1)
        #     client.loop()
    # [END iot_mqtt_run]

In [ ]:
mqtt_device_random(args)

Device client_id is 'projects/cmpe297-hw1/locations/us-central1/registries/CMPEIoT1/devices/cmpeiotdevice2'
Creating JWT using RS256 from private key file /content/iotdevicenew/rsa_private.pem
Subscribing to /devices/cmpeiotdevice2/commands/#
Publishing message 1/: '{"registry_id": "CMPEIoT1", "device_id": "cmpeiotdevice2", "timecollected": "2020-10-09 18:57:44", "zipcode": 95192, "latitude": "37.342085", "longitude": "-121.883054", "temperature": "21.67", "humidity": "71.63"}'
on_connect Connection Accepted.
Publishing message 2/: '{"registry_id": "CMPEIoT1", "device_id": "cmpeiotdevice2", "timecollected": "2020-10-09 18:57:45", "zipcode": 95192, "latitude": "37.345172", "longitude": "-121.884004", "temperature": "32.11", "humidity": "76.79"}'
Publishing message 3/: '{"registry_id": "CMPEIoT1", "device_id": "cmpeiotdevice2", "timecollected": "2020-10-09 18:57:46", "zipcode": 95192, "latitude": "37.341567", "longitude": "-121.876670", "temperature": "22.42", "humidity": "67.49"}'
Recei